In [20]:
import kagglehub

path = kagglehub.dataset_download("tushardobal/india-city-air-quality-index-dataset-20152023")
print(path)

C:\Users\OJASV\.cache\kagglehub\datasets\tushardobal\india-city-air-quality-index-dataset-20152023\versions\1


In [21]:
import os

csv_path=os.path.join(path,"india_city_aqi_2015_2023.csv")
print(csv_path)

C:\Users\OJASV\.cache\kagglehub\datasets\tushardobal\india-city-air-quality-index-dataset-20152023\versions\1\india_city_aqi_2015_2023.csv


In [26]:
import pandas as pd

df=pd.read_csv(csv_path)
print(df)

print()
print(df.columns)

            city        date        pm25        pm10        no2        so2  \
0          Delhi  2015-01-01   99.868566  147.103280  49.715328  19.615149   
1          Delhi  2015-01-02  143.168513  208.517207  32.957884  14.712800   
2          Delhi  2015-01-03   89.678491  101.412886  14.126233   9.188562   
3          Delhi  2015-01-04   43.679037   65.432963  61.984732  10.871118   
4          Delhi  2015-01-05   58.224691  110.443143  22.735096  13.878490   
...          ...         ...         ...         ...        ...        ...   
32865  Ahmedabad  2023-12-27  104.189767  125.996540  54.037235   9.458740   
32866  Ahmedabad  2023-12-28   78.296822  126.139390  21.102546  14.506215   
32867  Ahmedabad  2023-12-29  133.796319  174.428556  14.262469   9.569303   
32868  Ahmedabad  2023-12-30   34.293435   67.092247  43.458757  13.238309   
32869  Ahmedabad  2023-12-31   52.469690   82.606862  27.533347   7.257668   

             co         o3  aqi  aqi_category  
0      0.729754

In [27]:
# Removed the unwanted column
df=df.drop("date",axis=1)
df=df.drop("city",axis=1)
print(df.columns)

Index(['pm25', 'pm10', 'no2', 'so2', 'co', 'o3', 'aqi', 'aqi_category'], dtype='object')


In [28]:
# Target and feature seperation
y=df["aqi_category"]
X=df.drop("aqi_category",axis=1)

# Filling missing values
X=pd.get_dummies(X)


In [33]:
from sklearn.preprocessing import StandardScaler

# Scaling numeric values
scaler=StandardScaler()

X_scaled=scaler.fit_transform(X)

print(X)

             pm25        pm10        no2        so2        co         o3  aqi
0       99.868566  147.103280  49.715328  19.615149  0.729754  46.487946  103
1      143.168513  208.517207  32.957884  14.712800  0.660975  43.014054  141
2       89.678491  101.412886  14.126233   9.188562  0.496151  54.713710   82
3       43.679037   65.432963  61.984732  10.871118  0.820258  28.628777   50
4       58.224691  110.443143  22.735096  13.878490  0.619808  45.624594   69
...           ...         ...        ...        ...       ...        ...  ...
32865  104.189767  125.996540  54.037235   9.458740  1.214473  69.390920  102
32866   78.296822  126.139390  21.102546  14.506215  0.383592  45.033878   83
32867  133.796319  174.428556  14.262469   9.569303  1.058658  65.074983  127
32868   34.293435   67.092247  43.458757  13.238309  0.805538  27.954360   44
32869   52.469690   82.606862  27.533347   7.257668  0.636559  63.218900   60

[32870 rows x 7 columns]


In [40]:
# Train/Test 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_encoded=le.fit_transform(y)

X_train,X_test,y_train,y_test=train_test_split(X_scaled,y_encoded,test_size=0.2,random_state=42)

In [41]:
# Model
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(len(le.classes_), activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

print(model)

C:\Users\OJASV\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 64)             │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 772 (3.02 KB)

 Trainable params: 772 (3.02 KB)

 Non-trainable params: 0 (0.00 B)

<Sequential name=sequential_4, built=True>


In [42]:
# Train model

history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    verbose=1
)

Epoch 1/100
658/658 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9146 - loss: 0.3452 - val_accuracy: 0.9648 - val_loss: 0.1398
Epoch 2/100
658/658 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9747 - loss: 0.1145 - val_accuracy: 0.9844 - val_loss: 0.0824
Epoch 3/100
658/658 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9852 - loss: 0.0810 - val_accuracy: 0.9926 - val_loss: 0.0620
Epoch 4/100
658/658 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9896 - loss: 0.0655 - val_accuracy: 0.9909 - val_loss: 0.0525
Epoch 5/100
658/658 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9918 - loss: 0.0544 - val_accuracy: 0.9916 - val_loss: 0.0448
Epoch 6/100
658/658 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9921 - loss: 0.0468 - val_accuracy: 0.9928 - val_loss: 0.0411
Epoch 7/100
658/658 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9932 - loss: 0.0410 - val_accuracy: 0.9947 - val_loss: 0.0349
Epoch 8/100
658/658 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9935 - loss: 0.0366 - val_accu

In [46]:
loss, accuracy = model.evaluate(X_test, y_test)
print("\nTest Accuracy:", accuracy)

206/206 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9959 - loss: 0.0078

Test Accuracy: 0.9958928823471069


In [ ]:
import numpy as np

# Compare predictions vs actual
pred = model.predict(X_test)
pred_classes = np.argmax(pred, axis=1)

# Create comparison DataFrame
comparison = pd.DataFrame({
    'Predicted': le.inverse_transform(pred_classes),
    'Actual': le.inverse_transform(y_test)
})

print(comparison.head(20))
print(f"\nTotal correct: {(comparison['Predicted'] == comparison['Actual']).sum()} / {len(comparison)}")

206/206 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
       Predicted        Actual
0       Moderate      Moderate
1   Satisfactory  Satisfactory
2           Good          Good
3       Moderate      Moderate
4   Satisfactory  Satisfactory
5   Satisfactory  Satisfactory
6   Satisfactory  Satisfactory
7   Satisfactory  Satisfactory
8       Moderate      Moderate
9           Good          Good
10  Satisfactory  Satisfactory
11      Moderate      Moderate
12      Moderate      Moderate
13      Moderate      Moderate
14  Satisfactory  Satisfactory
15          Good          Good
16  Satisfactory  Satisfactory
17      Moderate      Moderate
18  Satisfactory  Satisfactory
19  Satisfactory  Satisfactory

Total correct: 6547 / 6574
